# K-Nearest Neighbor - Movie Review Classification

In [1]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords 
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer

ps = PorterStemmer()




#read input file
train_data = open("/home/ss/CMPE-255-Fa17/pr/train.dat","r")
ts = train_data.readlines()

test_data = open("/home/ss/CMPE-255-Fa17/pr/test.dat","r")
ts1 = test_data.readlines()




#train 
fl_len = len(ts)
cls = []
review = []


#test
ts_len = len(ts1)
t_rev = []


In [1]:
#This function is for finding nearest neighbor. It takes 'k' near neighbor and get sentiment for test data 
#based on majority sentiment obtained from train data

def FindNeighbor(inp_csr, K):
    output_sentiment = []
    for row in inp_csr:
        #get single row from input scr matrix
        row = np.array(row)
        #sort and get 'k' nearest neighbor indexes
        max_row = np.argpartition(-row, K)
        top_rows = max_row[:K]
        result = reviewCheck(top_rows, cls)
        output_sentiment.append(result)
    return output_sentiment

def reviewCheck(row_index, senti_mat):
    pos_senti = 0
    neg_senti = 0
    for index in row_index:
        #check for neighbor index with corresponding sentiment
        if senti_mat[int(index)] == "+1":
            pos_senti += 1
        else:
            neg_senti +=1
    #decide which would be sentiment of review 
    if pos_senti > neg_senti:
        return "+1"
    elif pos_senti < neg_senti:
        return "-1"

In [3]:

#data preprocessing for train data happens here. It removes html tags, get only alphabets, convert to lower case and 
#remove stop words 

for i in range(fl_len):
    line = ts[i]
    l_len = len(line)
    for t in range(l_len):
        if t ==  1:
            cls.append(line[0:2])
        else:
            if t == 2:
                text = BeautifulSoup(line[2:l_len], "lxml")
                letter = re.sub("[^a-zA-Z]", " ", text.get_text())
                l_case = letter.lower()
                words = l_case.split()
                #print words
                stop = set(stopwords.words("english"))
                m_words = [wr for wr in words if not wr in stop]
                
                text1 = (" ".join(m_words))
                review.append(text1)               

In [4]:

#data preprocessing for train data happens here. It removes html tags, get only alphabets, convert to lower case and 
#remove stop words 

t_rev = []
for i in range(ts_len):
    line = ts1[i]
    text = BeautifulSoup(line, "lxml")
    letter = re.sub("[^a-zA-Z]", " ", text.get_text())
    l_case = letter.lower()
    words = l_case.split()
    stop = set(stopwords.words("english"))
    m_words = [w for w in words if not w in stop]
    text1 = (" ".join(m_words))
    t_rev.append(text1) 

In [5]:

#create CSR matrix by implementing inverse document frequency and l2 normalization

from sklearn.feature_extraction.text import TfidfTransformer

vectorizer = TfidfVectorizer(norm = 'l2',min_df = 0, use_idf = True, smooth_idf = False, sublinear_tf = True, \
                             ngram_range=(1,2), max_features = 9000 )


train_vect = vectorizer.fit_transform(review)


train_vect = train_vect.toarray()
print train_vect.shape


(25000, 11000)


In [6]:
test_vect = vectorizer.transform(t_rev)

test_vect = test_vect.toarray()
print test_vect.shape


(25000, 11000)


In [7]:
#get cosine similarity
csr_sim = cosine_similarity(test_vect,train_vect)


In [107]:

#Nearest neighbor find

output = FindNeighbor(csr_sim,157)

with open('/home/ss/CMPE-255-Fa17/pr/format.dat', 'w') as log:
      for x in output:
          log.write(x+'\n')